# Importing Libraries

In [15]:
# Required imports
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kumar\AppData\Roaming\nltk_data...


True

# Loading the Intent Data

In [16]:
# Initialize lemmatizer and load the intents file
lemmatizer = WordNetLemmatizer()

# Load the intents JSON file
with open('intents.json') as file:
    intents = json.load(file)

# Initialize necessary variables
words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']


# Tokenizing and Lemmatizing the Patterns

In [17]:
# Tokenize and lemmatize the patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize and sort words and classes
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

# Save the words and classes using pickle
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))


# Preparing the Training Data

In [18]:
# Initialize training data
training = []
outputEmpty = [0] * len(classes)

# Create the training bag of words
for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

# Shuffle and convert training data into NumPy arrays
random.shuffle(training)
training = np.array(training)

# Split into input (trainX) and output (trainY)
trainX = training[:, :len(words)]
trainY = training[:, len(words):]


# Building and Training the Model

In [19]:
# Build the model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

# Compile the model
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


#  Training and Saving the Model

In [20]:
# Train the model
hist = model.fit(np.array(trainX), np.array(trainY), epochs=200, batch_size=5, verbose=1)

# Save the trained model
model.save('chatbot_model.h5', hist)

print('Model training complete.')


Epoch 1/200
4/4 [==============================] - 2s 9ms/step - loss: 1.4848 - accuracy: 0.1500
Epoch 2/200
4/4 [==============================] - 0s 5ms/step - loss: 1.4291 - accuracy: 0.3500
Epoch 3/200
4/4 [==============================] - 0s 5ms/step - loss: 1.4096 - accuracy: 0.2500
Epoch 4/200
4/4 [==============================] - 0s 6ms/step - loss: 1.2759 - accuracy: 0.4500
Epoch 5/200
4/4 [==============================] - 0s 5ms/step - loss: 1.3588 - accuracy: 0.4500
Epoch 6/200
4/4 [==============================] - 0s 7ms/step - loss: 1.1997 - accuracy: 0.5000
Epoch 7/200
4/4 [==============================] - 0s 5ms/step - loss: 1.2051 - accuracy: 0.5000
Epoch 8/200
4/4 [==============================] - 0s 6ms/step - loss: 1.1632 - accuracy: 0.4500
Epoch 9/200
4/4 [==============================] - 0s 7ms/step - loss: 1.0831 - accuracy: 0.6500
Epoch 10/200
4/4 [==============================] - 0s 6ms/step - loss: 1.0288 - accuracy: 0.6000
Epoch 11/200
4/4 [===========

4/4 [==============================] - 0s 5ms/step - loss: 0.0423 - accuracy: 1.0000
Epoch 85/200
4/4 [==============================] - 0s 4ms/step - loss: 0.0671 - accuracy: 0.9500
Epoch 86/200
4/4 [==============================] - 0s 5ms/step - loss: 0.1017 - accuracy: 0.9500
Epoch 87/200
4/4 [==============================] - 0s 8ms/step - loss: 0.0398 - accuracy: 1.0000
Epoch 88/200
4/4 [==============================] - 0s 6ms/step - loss: 0.0183 - accuracy: 1.0000
Epoch 89/200
4/4 [==============================] - 0s 6ms/step - loss: 0.0546 - accuracy: 1.0000
Epoch 90/200
4/4 [==============================] - 0s 6ms/step - loss: 0.0176 - accuracy: 1.0000
Epoch 91/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0348 - accuracy: 1.0000
Epoch 92/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0351 - accuracy: 1.0000
Epoch 93/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0424 - accuracy: 1.0000
Epoch 94/200
4/4 [===============

4/4 [==============================] - 0s 5ms/step - loss: 0.0083 - accuracy: 1.0000
Epoch 168/200
4/4 [==============================] - 0s 7ms/step - loss: 0.0168 - accuracy: 1.0000
Epoch 169/200
4/4 [==============================] - 0s 6ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 170/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0049 - accuracy: 1.0000
Epoch 171/200
4/4 [==============================] - 0s 6ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 172/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0068 - accuracy: 1.0000
Epoch 173/200
4/4 [==============================] - 0s 6ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 174/200
4/4 [==============================] - 0s 6ms/step - loss: 0.0106 - accuracy: 1.0000
Epoch 175/200
4/4 [==============================] - 0s 7ms/step - loss: 0.0065 - accuracy: 1.0000
Epoch 176/200
4/4 [==============================] - 0s 7ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 177/200
4/4 [=====

c:\users\kumar\appdata\local\programs\python\python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
